# **Classical Implementation**

Constants

In [3]:
"""Constants in common between the examples provided,
i.e., regardless of the framework used (whether PyTorch or
tf.keras)"""

BATCH_SIZE = 128

DROPOUT = 0.5

IMAGE_DIM = 28

KERNEL_SIZE_CONV = 3
KERNEL_SIZE_MAX_POOL = 2

NUM_CLASSES = 10
NUM_EPOCHS = 30

OUT_CHANNEL_CONV1 = 32
OUT_CHANNEL_CONV2 = 64

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms

inputs_shape = (1, IMAGE_DIM, IMAGE_DIM)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


class ConvNet(nn.Module):
    """
    Convolutional Neural Network model for image classification using classical ReLU activation.

    Args:
        modified: bool
            Whether to use the modified version of the QReLU or m-QReLU (default: False).

    Attributes:
        conv1: nn.Conv2d
            First convolutional layer.
        relu1: nn.ReLU
            First ReLU activation.
        pool1: nn.MaxPool2d
            First max pooling layer.
        conv2: nn.Conv2d
            Second convolutional layer.
        relu2: nn.ReLU
            Second ReLU activation.
        pool2: nn.MaxPool2d
            Second max pooling layer.
        flatten: nn.Flatten
            Flatten layer.
        dropout: nn.Dropout
            Dropout layer.
        fc: nn.Linear
            Fully connected layer.
    """

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, OUT_CHANNEL_CONV1, kernel_size=KERNEL_SIZE_CONV)
        self.relu1 = nn.ReLU()  # Using classical ReLU instead of QuantumReLU
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(OUT_CHANNEL_CONV1, OUT_CHANNEL_CONV2, kernel_size=KERNEL_SIZE_CONV)
        self.relu2 = nn.ReLU()  # Using classical ReLU instead of QuantumReLU
        self.pool2 = nn.MaxPool2d(kernel_size=KERNEL_SIZE_MAX_POOL)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(OUT_CHANNEL_CONV2 * 5 * 5, NUM_CLASSES)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network.

        Args:
            x: torch.Tensor
                The Input tensor.

        Returns:
            Output tensor (torch.Tensor).
        """
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x


def train(
        model: nn.Module,
        train_loader: DataLoader,
        optimizer: optim.Optimizer,
        criterion: nn.Module,
        epochs: int = NUM_EPOCHS
) -> None:
    """
    Train the model.

    Args:
        model: nn.Module
            The neural network model to be trained.
        train_loader: DataLoader
            Data loader for training data.
        optimizer: optim.Optimizer
            Optimizer for training.
        criterion: nn.Module
            Loss function.
        epochs: int
            Number of epochs for training (default: NUM_EPOCHS).
    """
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")


def test(
        model: nn.Module,
        test_loader: DataLoader
) -> None:
    """
    Evaluate the trained model on test data.

    Args:
        model: nn.Module
            The trained model to be evaluated.
        test_loader: DataLoader
            Data loader for test data.
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")


# Initialize the model
model = ConvNet()
print(model)

# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train and test the modelm
train(model, train_loader, optimizer, criterion)
test(model, test_loader)

ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1600, out_features=10, bias=True)
)
Epoch [1/30], Loss: 0.2351
Epoch [2/30], Loss: 0.0833
Epoch [3/30], Loss: 0.0656
Epoch [4/30], Loss: 0.0548
Epoch [5/30], Loss: 0.0493
Epoch [6/30], Loss: 0.0447
Epoch [7/30], Loss: 0.0427
Epoch [8/30], Loss: 0.0388
Epoch [9/30], Loss: 0.0369
Epoch [10/30], Loss: 0.0350
Epoch [11/30], Loss: 0.0330
Epoch [12/30], Loss: 0.0326
Epoch [13/30], Loss: 0.0286
Epoch [14/30], Loss: 0.0293
Epoch [15/30], Loss: 0.0261
Epoch [16/30], Loss: 0.0272
Epoch [17/30], Loss: 0.0253
Epoch [18/30], Loss: 0.0237
Epoch [19/30], 

# **Quantum Implementation**

Constants

In [ ]:
"""Constants in common between the examples provided,
i.e., regardless of the framework used (whether PyTorch or
tf.keras)"""

BATCH_SIZE = 128

DROPOUT = 0.5

IMAGE_DIM = 28

KERNEL_SIZE_CONV = 3
KERNEL_SIZE_MAX_POOL = 2

NUM_CLASSES = 10
NUM_EPOCHS = 2

OUT_CHANNEL_CONV1 = 32
OUT_CHANNEL_CONV2 = 64

Src.Constants

In [ ]:
"""
This file contains constants leveraged across the two quantum activation functions
QReLU and m-QReLU regardless of the framework (TensorFlow or Keras or PyTorch)
of their implementation.
"""

import torch

FIRST_COEFFICIENT = 0.01
SECOND_COEFFICIENT_QRELU = 2
SECOND_COEFFICIENT_M_QRELU = 1

FIRST_COEFFICIENT_PYTORCH = torch.tensor(FIRST_COEFFICIENT)
SECOND_COEFFICIENT_QRELU_PYTORCH = torch.tensor(SECOND_COEFFICIENT_QRELU)
SECOND_COEFFICIENT_M_QRELU_PYTORCH = torch.tensor(SECOND_COEFFICIENT_M_QRELU)

M_QRELU_NAME = "modified_quantum_relu"
QRELU_NAME = "quantum_relu"

USE_M_QRELU = False

pytorch.quantum activations

In [ ]:
"""Additional utility for Deep Learning models in PyTorch"""

# The Quantum ReLU (QReLU) and the modified QReLU (m-QReLU) as custom activation functions in
# PyTorch

# Author: Luca Parisi <luca.parisi@ieee.org>

import torch
import torch.nn as nn
# from src.constants import (FIRST_COEFFICIENT_PYTORCH, M_QRELU_NAME, QRELU_NAME,
#                            SECOND_COEFFICIENT_M_QRELU_PYTORCH,
#                            SECOND_COEFFICIENT_QRELU_PYTORCH, USE_M_QRELU)


class QuantumReLU(nn.Module):  # pragma: no cover
    """
    A class defining the QuantumReLU activation function in PyTorch.
    """

    def __init__(self, modified: bool = USE_M_QRELU) -> None:
        """
        Initialise the QuantumReLU activation function.

        Args:
            modified: bool
                    Whether using the modified version of the QReLU or m-QReLU (no/False by default,
                    i.e., using the QReLU by default).
        """

        self.modified = modified
        self._name = M_QRELU_NAME if modified else QRELU_NAME
        super().__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Call the QuantumReLU activation function.

        Args:
            x: torch.Tensor
                The input tensor.

        Returns:
                The output tensor (torch.Tensor) from the QuantumReLU activation function.
        """
        return torch_quantum_relu(x=x, modified=self.modified)


def torch_quantum_relu(x: torch.Tensor, modified: bool = USE_M_QRELU) -> torch.Tensor:
    """
    Apply the QReLU activation function or its modified version (m-QReLU) to transform inputs accordingly.

    Args:
        x: torch.Tensor
            The input tensor to be transformed via the m-QReLU activation function.
        modified: bool
            Whether using the modified version of the QReLU or m-QReLU (no/False by default, i.e.,
            using the QReLU by default).

    Returns:
            The transformed x (torch.Tensor) via the QReLU or m-QReLU.
    """
    second_coefficient = SECOND_COEFFICIENT_M_QRELU_PYTORCH if modified else SECOND_COEFFICIENT_QRELU_PYTORCH
    return torch.where(
            x <= 0,
            FIRST_COEFFICIENT_PYTORCH * x - second_coefficient * x,
            x,
    )

In [ ]:
"""
Example of a simple MNIST image classifier using the QReLU or m-QReLU activation function in its two
convolutional layers.

Adapted from https://keras.io/examples/vision/mnist_convnet/
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
# from constants import (BATCH_SIZE, DROPOUT, IMAGE_DIM, KERNEL_SIZE_CONV,
#                        KERNEL_SIZE_MAX_POOL, NUM_CLASSES, NUM_EPOCHS,
#                        OUT_CHANNEL_CONV1, OUT_CHANNEL_CONV2)
# from src.constants import USE_M_QRELU
# from src.pytorch.quantum_activations import QuantumReLU
from torch.utils.data import DataLoader
from torchvision import transforms

inputs_shape = (1, IMAGE_DIM, IMAGE_DIM)

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


class ConvNet(nn.Module):
    """
    Convolutional Neural Network model for image classification.

    Args:
        modified: bool
            Whether to use the modified version of the
            QReLU or m-QReLU (default: False).

    Attributes:
        conv1: nn.Conv2d
            First convolutional layer.
        relu1: QuantumReLU
            First Quantum ReLU activation.
        pool1: nn.MaxPool2d
            First max pooling layer.
        conv2: nn.Conv2d
            Second convolutional layer.
        relu2: QuantumReLU
            Second Quantum ReLU activation.
        pool2: nn.MaxPool2d
            Second max pooling layer.
        flatten: nn.Flatten
            Flatten layer.
        dropout: nn.Dropout
            Dropout layer.
        fc: nn.Linear
            Fully connected layer.
    """

    def __init__(self, modified=False):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, OUT_CHANNEL_CONV1,
                               kernel_size=KERNEL_SIZE_CONV)
        self.relu1 = QuantumReLU(modified=modified)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(
            OUT_CHANNEL_CONV1, OUT_CHANNEL_CONV2, kernel_size=KERNEL_SIZE_CONV)
        self.relu2 = QuantumReLU(modified=modified)
        self.pool2 = nn.MaxPool2d(kernel_size=KERNEL_SIZE_MAX_POOL)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(OUT_CHANNEL_CONV2 * 5 * 5, NUM_CLASSES)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network.

        Args:
            x: torch.Tensor
                The Input tensor.

        Returns:
            Output tensor (torch.Tensor).
        """
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x


def train(
        model: nn.Module,
        train_loader: DataLoader,
        optimizer: optim.Optimizer,
        criterion: nn.Module,
        epochs: int = NUM_EPOCHS
) -> None:
    """
    Train the model.

    Args:
        model: nn.Module
            The neural network model to be trained.
        train_loader: DataLoader
            Data loader for training data.
        optimizer: optim.Optimizer
            Optimizer for training.
        criterion: nn.Module
            Loss function.
        epochs: int
            Number of epochs for training (default: NUM_EPOCHS).
    """
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")


def test(
        model: nn.Module,
        test_loader: DataLoader
) -> None:
    """
    Evaluate the trained model on test data.

    Args:
        model: nn.Module
            The trained model to be evaluated.
        test_loader: DataLoader
            Data loader for test data.
    """
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")


model = ConvNet(modified=USE_M_QRELU)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train(model, train_loader, optimizer, criterion)
test(model, test_loader)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 488kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.41MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.97MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): QuantumReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu2): QuantumReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1600, out_features=10, bias=True)
)
Epoch [1/2], Loss: 0.3163
Epoch [2/2], Loss: 0.1237
Accuracy: 98.22%
